In [2]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
import json
import ast

In [3]:
df = pd.read_csv('data/json_to_df_result.csv')
for col in ['annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)
df.head(3)

,source_type,annotations,equipment,filename,id,gender,age,date,skin_type,sensitive,device,width,height,angle,facepart,bbox
0,Validation,{'acne': 0},{'pigmentation_count': 147},0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,0,"[0, 0, 2136, 3216]"
1,Validation,"{'forehead_pigmentation': 1, 'forehead_wrinkle...","{'forehead_moisture': 53.0, 'forehead_elastici...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,1,"[483, 916, 1641, 1452]"
2,Validation,{'glabellus_wrinkle': 1},{},0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,2,"[955, 1426, 1243, 1769]"


In [4]:
df1 = df.drop(columns=['source_type'])
df1 = df1[df1['facepart'] != 0]
df1.head(3)

,annotations,equipment,filename,id,gender,age,date,skin_type,sensitive,device,width,height,angle,facepart,bbox
1,"{'forehead_pigmentation': 1, 'forehead_wrinkle...","{'forehead_moisture': 53.0, 'forehead_elastici...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,1,"[483, 916, 1641, 1452]"
2,{'glabellus_wrinkle': 1},{},0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,2,"[955, 1426, 1243, 1769]"
3,{'l_perocular_wrinkle': 3},"{'l_perocular_wrinkle_Ra': 14.581, 'l_perocula...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,3,"[198, 1598, 414, 2060]"


In [5]:
x = df1['id'].unique()
train, test = train_test_split(x, test_size=0.25,random_state=0)
val, test = train_test_split(test, test_size=0.5, random_state=0)
len(train), len(test), len(val)

(723, 121, 121)

In [6]:
sorted_numbers = sorted(list(test))
print(sorted_numbers)

[3, 4, 10, 22, 42, 43, 46, 61, 65, 90, 121, 123, 135, 136, 161, 167, 183, 187, 189, 197, 198, 204, 208, 210, 219, 225, 239, 252, 263, 265, 272, 299, 308, 320, 332, 335, 350, 351, 357, 362, 377, 395, 404, 406, 422, 427, 434, 444, 466, 483, 505, 511, 527, 537, 540, 544, 549, 584, 589, 596, 599, 611, 614, 616, 619, 633, 650, 659, 661, 679, 686, 701, 709, 719, 725, 727, 733, 734, 742, 745, 753, 759, 780, 784, 794, 796, 798, 816, 819, 823, 831, 832, 834, 844, 861, 865, 867, 879, 889, 919, 921, 926, 927, 931, 955, 967, 976, 983, 987, 1003, 1005, 1017, 1036, 1037, 1054, 1056, 1086, 1093, 1096, 1098, 1100]


In [10]:
df.to_csv('final.csv', index=False)

In [152]:
# train/val/test 분할
x = df1['id'].unique()
train, test = train_test_split(x, test_size=0.25,random_state=0)
val, test = train_test_split(test, test_size=0.5, random_state=0)

# 분할된 데이터프레임에 split 열 추가
train = df1[df1['id'].isin(list(train))]
val = df1[df1['id'].isin(list(val))]
test = df1[df1['id'].isin(list(test))]

train['source_type'] = 'train'
val['source_type'] = 'val'
test['source_type'] = 'test'

# 데이터프레임 합치기
df2 = pd.concat([train, val])
df = pd.concat([df2, test])

# 기본 경로 설정
base_path = 'data/Training_cropped'
cls_base_path = 'data/classified_cropped'

# 이미지 복사 및 분류 함수
def copy_and_classify_image(row):
    facepart = row['facepart']
    filename = row['filename'].replace('.jpg', f'_{facepart}.jpg')
    src_path = os.path.join(base_path, str(facepart), filename)
    
    if not os.path.exists(src_path):
        print(f"Warning: Source file not found: {src_path}")
        return

    annotations = row['annotations']
    
    for key, value in annotations.items():
        cls_path = os.path.join(cls_base_path, key, row['source_type'], str(value))
        os.makedirs(cls_path, exist_ok=True)
        cls_file = os.path.join(cls_path, filename)
        shutil.copy2(src_path, cls_file)

# 각 행에 대해 이미지 복사 및 분류 함수 적용
df.apply(copy_and_classify_image, axis=1)

print("이미지 분류 및 복사가 완료되었습니다.")

C:\Users\gillhk\AppData\Local\Temp\ipykernel_19172\2997729297.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['source_type'] = 'train'
C:\Users\gillhk\AppData\Local\Temp\ipykernel_19172\2997729297.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['source_type'] = 'val'
C:\Users\gillhk\AppData\Local\Temp\ipykernel_19172\2997729297.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

이미지 분류 및 복사가 완료되었습니다.


In [2]:
import shutil
import os

def compress_to_zip(folder_path, output_zip):
    shutil.make_archive(output_zip, 'zip', folder_path)
    print(f"{folder_path}가 {output_zip}.zip으로 압축되었습니다.")

# 사용 예제
folder_path = 'data/classified_cropped'  # 압축할 폴더 경로
output_zip = 'data/classified_cropped_image'  # 생성될 ZIP 파일 경로 (확장자 제외)

compress_to_zip(folder_path, output_zip)

data/classified_cropped가 data/classified_cropped_image.zip으로 압축되었습니다.


In [1]:
df

NameError: name 'df' is not defined